In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import math

from scipy.stats import ks_2samp

import joblib

import dice_ml
from dice_ml.utils import helpers # helper functions

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)


## Cargar Modelos

In [2]:
# import os
# import joblib

# # Ruta de la carpeta donde se guardaron los modelos
# carpeta_modelos = "Modelos_2"

# # Cargar los modelos
# random_forest_model = joblib.load(os.path.join(carpeta_modelos, 'RandomForest_model.pkl'))
# gradient_boosting_model = joblib.load(os.path.join(carpeta_modelos, 'GradientBoosting_model.pkl'))
# adaboost_model = joblib.load(os.path.join(carpeta_modelos, 'AdaBoost_model.pkl'))

# # Verificar que los modelos se cargaron correctamente
# print("Modelos cargados correctamente:")
# print("Random Forest:", random_forest_model)
# print("Gradient Boosting:", gradient_boosting_model)
# print("AdaBoost:", adaboost_model)


In [3]:
knn_model = joblib.load('Modelos/K-NearestNeighborsClas.joblib')

In [4]:
knn_model.n_features_in_

12

In [9]:
knn_model

KNeighborsClassifier(leaf_size=15, n_neighbors=9)

In [8]:
knn_model_2 = joblib.load('Modelos_2/AdaBoostClas.joblib')
#extra_trees_model = joblib.load('Modelos/ExtraTreesReg.joblib')

In [10]:
knn_model_2

AdaBoostClassifier(algorithm='SAMME', learning_rate=0.1, n_estimators=200,
                   random_state=42)

In [11]:


RandomForestReg_NUEVO = joblib.load('Modelos_2/RandomForestReg_NUEVO.joblib')

In [57]:
RandomForestReg_NUEVO.feature_names_in_

array(['Alim_CuT', 'Alim_CuS', 'Alim CuI', 'Ag', 'Pb', 'Fe',
       'P80_Alim_Ro300', 'pH_Ro300', 'Tratamiento_Turno', 'Sol_Cit',
       'Aire_Celdas', 'Nivel_Celdas'], dtype=object)

## Carga de Datos

In [58]:
# Carga el archivo Excel en un DataFrame
df = pd.read_excel('df_capstone.xlsx')
df['cumplimiento'] = np.where(df['Recuperacion_Turno'] >= 86, 1, 0)
df=df.drop(columns=['Fecha','Turno'])#,'Alim CuI'])
Variables_predictoras = RandomForestReg_NUEVO.feature_names_in_
Variables_ajustables = ['P80_Alim_Ro300', 'Tratamiento_Turno', 'pH_Ro300','Aire_Celdas','Nivel_Celdas']
Variables_Objetivo = ['Recuperacion_Turno','cumplimiento']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Alim_CuT            600 non-null    float64
 1   Alim_CuS            600 non-null    float64
 2   Alim CuI            600 non-null    float64
 3   Ag                  600 non-null    float64
 4   Pb                  600 non-null    float64
 5   Fe                  600 non-null    float64
 6   Recuperacion_Turno  600 non-null    float64
 7   P80_Alim_Ro300      600 non-null    float64
 8   pH_Ro300            600 non-null    float64
 9   Tratamiento_Turno   600 non-null    float64
 10  Sol_Cit             600 non-null    float64
 11  Aire_Celdas         600 non-null    float64
 12  Nivel_Celdas        600 non-null    float64
 13  cumplimiento        600 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 65.8 KB


## separación y estandarización de datos

In [59]:
## Características de entrenamiento para clasificación
X_clas = df[Variables_predictoras]
X_reg = df[Variables_predictoras]
## Etiquetas / clases
y_clas = df['cumplimiento']
y_reg = df['Recuperacion_Turno']

scaler_data = StandardScaler()
X_clas = scaler_data.fit_transform(X_clas)
dataset = pd.DataFrame(X_clas, columns=Variables_predictoras)

dataset['cumplimiento'] = y_clas
# Mover la columna 'cumplimiento' al primer lugar
columnas = ['cumplimiento'] + [col for col in dataset.columns if col != 'cumplimiento']
dataset = dataset[columnas]

# Dividir el dataset en entrenamiento y prueba
train_dataset, test_dataset = train_test_split(dataset,test_size=0.2,random_state=42, stratify=dataset['cumplimiento'])

display(train_dataset.head())

# Verificar las proporciones de la columna 'cumplimiento'
print("Proporciones en el conjunto de entrenamiento:")
print(train_dataset['cumplimiento'].value_counts(normalize=True))
print("\nProporciones en el conjunto de prueba:")
print(test_dataset['cumplimiento'].value_counts(normalize=True))

,cumplimiento,Alim_CuT,Alim_CuS,Alim CuI,Ag,Pb,Fe,P80_Alim_Ro300,pH_Ro300,Tratamiento_Turno,Sol_Cit,Aire_Celdas,Nivel_Celdas
571,1,0.843584,-0.818165,1.016330,-0.867609,5.857320,1.850925,-1.805305,-0.075614,-1.322124,-1.217751,-0.000030,-1.353332
423,0,0.575268,-0.489365,0.681429,0.491156,0.064516,0.121079,-0.909156,-0.271898,0.450405,-0.375551,-0.458569,0.273638
160,0,-0.766309,0.825837,-0.937256,-0.718145,-0.272725,-0.198467,0.086564,-0.249465,-0.577685,1.018290,0.464856,0.879066
331,1,0.253290,0.168236,0.234895,0.038234,2.710811,0.843432,1.107178,0.137493,0.823064,-0.796651,-0.100801,-1.411553
461,0,-1.732245,-1.146965,-1.607057,-1.320531,-0.919776,-1.380656,-0.187259,0.928234,-0.691275,-1.217751,1.270919,0.574691


Proporciones en el conjunto de entrenamiento:
cumplimiento
0    0.7625
1    0.2375
Name: proportion, dtype: float64

Proporciones en el conjunto de prueba:
cumplimiento
0    0.766667
1    0.233333
Name: proportion, dtype: float64


## Estadísticas de las variables predictoras

In [60]:
# Crear el DataFrame con medias y desviaciones estándar
statistics_df = pd.DataFrame(
    data={
        "mean": scaler_data.mean_,  # Medias de cada característica
        "std": scaler_data.scale_  # Desviaciones estándar de cada característica
    },
    index=Variables_predictoras  # Nombres de las variables predictoras como índice
)

# Mostrar el DataFrame con medias y desviaciones estándar
statistics_df.reset_index(inplace=True)
statistics_df.rename(columns={'index':'Variables'}, inplace=True)
statistics_df

,Variables,mean,std
0,Alim_CuT,0.792800,0.186348
1,Alim_CuS,0.084883,0.030414
2,Alim CuI,0.707917,0.179158
3,Ag,5.915583,2.207888
4,Pb,94.001700,61.973450
5,Fe,2.716305,0.526058
6,P80_Alim_Ro300,216.522550,40.171915
7,pH_Ro300,10.848277,0.594379
8,Tratamiento_Turno,7951.510162,1963.212406
9,Sol_Cit,0.068918,0.023747


## Modelo Contrafactual

### definición de modelo contrafactual

In [61]:
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['Alim_CuT', 'Alim_CuS', 'Ag', 'Pb', 'Fe', 'P80_Alim_Ro300',
       'pH_Ro300', 'Tratamiento_Turno', 'Sol_Cit', 'Aire_Celdas','Nivel_Celdas'],
                 outcome_name='cumplimiento')

m = dice_ml.Model(model=knn_model, backend="sklearn") # se utiliza un modelo de clasificación para el modelo contrafactual
exp = dice_ml.Dice(d, m, method="random")

In [62]:
# Verificar las variables en train_dataset
print("Columnas en train_dataset:", train_dataset.columns)
print("Número de columnas en train_dataset:", train_dataset.shape[1])

Columnas en train_dataset: Index(['cumplimiento', 'Alim_CuT', 'Alim_CuS', 'Alim CuI', 'Ag', 'Pb', 'Fe',
       'P80_Alim_Ro300', 'pH_Ro300', 'Tratamiento_Turno', 'Sol_Cit',
       'Aire_Celdas', 'Nivel_Celdas'],
      dtype='object')
Número de columnas en train_dataset: 13


In [63]:
# import pandas as pd
# import numpy as np

# # Crear copia del dataframe original
# df_resultado = df.copy()
# df_resultado['encontro_contrafactual'] = 'no'  # Nueva columna, por defecto 'no'

# # Filtrar registros donde 'cumplimiento == 0'
# registros = df.query('cumplimiento == 0')

# # Crear arrays de costos
# df_costos = pd.DataFrame({
#     "Variables": ["P80_Alim_Ro300", "Tratamiento_Turno", "pH_Ro300", "Aire_Celdas", "Nivel_Celdas"],
#     "Costo_Bajar": [0.1, 0.8, 1.5, 0.2, 0.5],
#     "Costo_Subir": [0.2, 0.9, 2, 0.4, 0.7]
# })
# costos_bajar = df_costos['Costo_Bajar'].values
# costos_subir = df_costos['Costo_Subir'].values


# # Iterar sobre cada registro
# for idx, registro in registros.iterrows():
#     try:
#         # Preparar el registro eliminando la columna objetivo
#         registro_sin_objetivo = registro.drop(['cumplimiento']).to_frame().T
#         registro_original = registro_sin_objetivo.copy()
#         registro_sin_objetivo = registro_sin_objetivo[Variables_predictoras]

#         # Escalar el registro
#         registro_scaled = scaler_data.transform(registro_sin_objetivo)
#         registro_scaled = pd.DataFrame(registro_scaled, columns=Variables_predictoras)

#         # Generar contrafactuales
#         e = exp.generate_counterfactuals(
#             registro_scaled,
#             total_CFs=20,
#             desired_class="opposite",
#             features_to_vary=Variables_ajustables,
#             verbose=False
#         )

#         # Verificar si se encontraron contrafactuales
#         if e.cf_examples_list[0].final_cfs_df.shape[0] > 0:
#             # Obtener los contrafactuales desescalados
#             c = e.cf_examples_list[0].final_cfs_df
#             df_contrafactual = c[Variables_ajustables]
#             df_contrafactual = (df_contrafactual * statistics_df[statistics_df['Variables']
#                                 .isin(Variables_ajustables)]['std'].values + 
#                                 statistics_df[statistics_df['Variables']
#                                 .isin(Variables_ajustables)]['mean'].values)

#             # Calcular variaciones absolutas
#             variacion_absoluta = df_contrafactual - registro_original[Variables_ajustables].values

#             # Calcular costos asociados
#             resultado = pd.DataFrame(index=variacion_absoluta.index, columns=variacion_absoluta.columns)
#             for i, column in enumerate(variacion_absoluta.columns):
#                 resultado[column] = np.where(
#                     variacion_absoluta[column] < 0,
#                     variacion_absoluta[column] * costos_bajar[i],
#                     variacion_absoluta[column] * costos_subir[i]
#                 )
#             resultado['Costo_Total'] = resultado.sum(axis=1)

#             # Seleccionar el contrafactual con el menor costo
#             indice_menor_costo = abs(resultado['Costo_Total']).idxmax()
#             mejor_contrafactual = df_contrafactual.loc[indice_menor_costo]



#             # Actualizar el DataFrame con el mejor contrafactual
#             df_resultado.loc[idx, Variables_ajustables] = mejor_contrafactual.values
#             df_resultado.loc[idx, 'encontro_contrafactual'] = 'si'
#         else:
#             print(f"No se encontraron contrafactuales para índice: {idx}")

#     except Exception as e:
#         print(f"Error al procesar índice {idx}: {e}")
#         continue

# # Mostrar el DataFrame resultante
# print("Proceso completado. Resultados:")
# display(df_resultado)


In [64]:
print("Columnas de df:", df.columns)
print("Feature names del modelo:", RandomForestReg_NUEVO.feature_names_in_)
print("Variables ajustables verificadas:", Variables_ajustables)
print("Estadísticas disponibles en statistics_df:")
print(statistics_df.head())

Columnas de df: Index(['Alim_CuT', 'Alim_CuS', 'Alim CuI', 'Ag', 'Pb', 'Fe',
       'Recuperacion_Turno', 'P80_Alim_Ro300', 'pH_Ro300', 'Tratamiento_Turno',
       'Sol_Cit', 'Aire_Celdas', 'Nivel_Celdas', 'cumplimiento'],
      dtype='object')
Feature names del modelo: ['Alim_CuT' 'Alim_CuS' 'Alim CuI' 'Ag' 'Pb' 'Fe' 'P80_Alim_Ro300'
 'pH_Ro300' 'Tratamiento_Turno' 'Sol_Cit' 'Aire_Celdas' 'Nivel_Celdas']
Variables ajustables verificadas: ['P80_Alim_Ro300', 'Tratamiento_Turno', 'pH_Ro300', 'Aire_Celdas', 'Nivel_Celdas']
Estadísticas disponibles en statistics_df:
  Variables       mean        std
0  Alim_CuT   0.792800   0.186348
1  Alim_CuS   0.084883   0.030414
2  Alim CuI   0.707917   0.179158
3        Ag   5.915583   2.207888
4        Pb  94.001700  61.973450


In [65]:
print("Valores nulos en Variables_ajustables:")
print(df[Variables_ajustables].isnull().sum())

Valores nulos en Variables_ajustables:
P80_Alim_Ro300       0
Tratamiento_Turno    0
pH_Ro300             0
Aire_Celdas          0
Nivel_Celdas         0
dtype: int64


In [66]:
# Revisar columnas esperadas por el escalador
print("Columnas esperadas por scaler_data:", scaler_data.feature_names_in_)

# Reordenar y seleccionar columnas para el escalador
ejemplo = df[scaler_data.feature_names_in_].iloc[:1]
print("Ejemplo reordenado para el escalador:")
print(ejemplo)

# Escalar el ejemplo
try:
    ejemplo_escalado = scaler_data.transform(ejemplo)
    print("Escalado exitoso:", ejemplo_escalado)
except Exception as e:
    print(f"Error durante el escalado: {e}")


Columnas esperadas por scaler_data: ['Alim_CuT' 'Alim_CuS' 'Alim CuI' 'Ag' 'Pb' 'Fe' 'P80_Alim_Ro300'
 'pH_Ro300' 'Tratamiento_Turno' 'Sol_Cit' 'Aire_Celdas' 'Nivel_Celdas']
Ejemplo reordenado para el escalador:
   Alim_CuT  Alim_CuS  Alim CuI   Ag    Pb    Fe  P80_Alim_Ro300  pH_Ro300  \
0       0.7      0.08      0.62  5.0  23.0  2.61           209.0      10.4   

   Tratamiento_Turno  Sol_Cit  Aire_Celdas  Nivel_Celdas  
0        7827.943419     0.05   109.206438     18.555931  
Escalado exitoso: [[-0.4979936  -0.16056419 -0.49072214 -0.41468746 -1.14567932 -0.202079
  -0.18725893 -0.75419358 -0.0629411  -0.79665105  0.19884329 -0.27067785]]


In [67]:
import pandas as pd
import numpy as np

# Crear copia del DataFrame original
df_resultado = df.copy()
df_resultado['encontro_contrafactual'] = 'no'  # Nueva columna, por defecto 'no'

# Filtrar registros donde 'cumplimiento == 0'
registros = df.query('cumplimiento == 0')

# Iterar sobre cada registro
for idx, registro in registros.iterrows():
    try:
        # Preparar registro eliminando la columna objetivo
        registro_sin_objetivo = registro.drop(['cumplimiento']).to_frame().T
        registro_sin_objetivo = registro_sin_objetivo[scaler_data.feature_names_in_]

        # Escalar el registro
        registro_scaled = scaler_data.transform(registro_sin_objetivo)
        registro_scaled = pd.DataFrame(registro_scaled, columns=scaler_data.feature_names_in_)

        # Generar contrafactuales
        e = exp.generate_counterfactuals(
            query_instances=registro_scaled,
            total_CFs=50,
            desired_class=1,  # Clase objetivo deseada
            features_to_vary=Variables_ajustables,
            verbose=True
        )

        # Verificar si se encontraron contrafactuales
        if e.cf_examples_list[0].final_cfs_df.shape[0] > 0:
            # Obtener los contrafactuales desescalados
            c = e.cf_examples_list[0].final_cfs_df
            df_contrafactual = c[Variables_ajustables]
            df_contrafactual = (df_contrafactual *
                                statistics_df.set_index('Variables').loc[Variables_ajustables, 'std'].values +
                                statistics_df.set_index('Variables').loc[Variables_ajustables, 'mean'].values)

            # Crear DataFrame completo con todas las columnas esperadas
            df_contrafactual_completo = pd.DataFrame(
                np.tile(registro_sin_objetivo.values, (df_contrafactual.shape[0], 1)),
                columns=registro_sin_objetivo.columns
            )

            # Reemplazar las columnas ajustadas con valores contrafactuales
            for col in Variables_ajustables:
                df_contrafactual_completo[col] = df_contrafactual[col].values

            # Asegurar el orden de columnas esperado por el modelo
            df_contrafactual_completo = df_contrafactual_completo[scaler_data.feature_names_in_]

            # Predecir la recuperación para cada contrafactual
            df_contrafactual['recuperacion_predicha'] = RandomForestReg_NUEVO.predict(df_contrafactual_completo)

            # Seleccionar el contrafactual con la máxima recuperación predicha
            mejor_contrafactual = df_contrafactual.loc[df_contrafactual['recuperacion_predicha'].idxmax()]

            # Actualizar el DataFrame resultado con el mejor contrafactual
            df_resultado.loc[idx, Variables_ajustables] = mejor_contrafactual[Variables_ajustables].values
            df_resultado.loc[idx, 'encontro_contrafactual'] = 'si'
            df_resultado.loc[idx, 'recuperacion_predicha'] = mejor_contrafactual['recuperacion_predicha']
        else:
            print(f"No se encontraron contrafactuales para índice: {idx}")

    except Exception as e:
        print(f"Error al procesar índice {idx}: {e}")
        continue

# Mostrar el DataFrame resultante
print("Proceso completado. Resultados:")
display(df_resultado)


100%|██████████| 1/1 [00:00<00:00, 10.70it/s]


No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec
Error al procesar índice 0: No counterfactuals found for any of the query points! Kindly check your configuration.


100%|██████████| 1/1 [00:00<00:00, 10.55it/s]


No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec
Error al procesar índice 1: No counterfactuals found for any of the query points! Kindly check your configuration.


100%|██████████| 1/1 [00:00<00:00,  8.71it/s]


...

100%|██████████| 1/1 [01:21<00:00, 81.76s/it]


Diverse Counterfactuals found! total time taken: 01 min 21 sec


100%|██████████| 1/1 [00:09<00:00,  9.13s/it]

Diverse Counterfactuals found! total time taken: 00 min 09 sec
Proceso completado. Resultados:


,Alim_CuT,Alim_CuS,Alim CuI,Ag,Pb,Fe,Recuperacion_Turno,P80_Alim_Ro300,pH_Ro300,Tratamiento_Turno,Sol_Cit,Aire_Celdas,Nivel_Celdas,cumplimiento,encontro_contrafactual,recuperacion_predicha
0,0.70,0.08,0.62,5.0,23.0,2.61,79.002079,209.0,10.400000,7827.943419,0.0500,109.206438,18.555931,0,no,NaN
1,0.64,0.08,0.56,4.0,29.0,2.72,78.174603,228.0,11.200000,7863.186581,0.0800,109.041660,21.650116,0,no,NaN
2,0.78,0.09,0.69,4.0,42.0,2.99,82.123510,198.0,11.343029,6781.529837,0.0800,95.063069,13.064303,0,si,80.275602
3,0.60,0.07,0.53,5.0,43.0,2.80,90.048983,183.0,11.500000,6011.236369,0.0600,117.918102,28.498127,1,no,NaN
4,0.67,0.09,0.58,4.0,36.0,2.78,84.133993,218.0,11.200000,6420.382068,0.0700,118.092912,28.460662,0,no,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.91,0.07,0.84,6.0,120.0,3.67,87.775226,243.0,10.780000,9459.450000,0.0635,106.473936,9.634823,1,no,NaN
596,0.94,0.09,0.85,5.0,71.0,3.60,90.174058,247.0,10.566667,7122.060000,0.0722,108.505663,15.414668,1,no,NaN
597,0.72,0.06,0.66,6.0,65.0,2.76,86.434413,208.0,11.093333,8291.250000,0.0560,104.836123,14.108598,1,no,NaN
598,0.80,0.06,0.74,5.0,228.0,3.19,89.483231,226.0,10.850000,7264.620000,0.0596,107.879839,13.586933,1,no,NaN


In [68]:
# Comparar la predicción del modelo antes y después
print("\nEvaluando el modelo Extra Trees:")

# Predecir con el DataFrame original
y_pred_antes = RandomForestReg_NUEVO.predict(df[Variables_predictoras])
df_resultado['prediccion_antes'] = y_pred_antes

# Predecir con el DataFrame ajustado
y_pred_despues = RandomForestReg_NUEVO.predict(df_resultado[Variables_predictoras])
df_resultado['prediccion_despues'] = y_pred_despues

# Calcular mejora absoluta
df_resultado['mejora_absoluta'] = df_resultado['prediccion_despues'] - df_resultado['prediccion_antes']

# Comparar resultados
mejora = (df_resultado['mejora_absoluta'] > 0).sum()
total = len(df_resultado['prediccion_antes'])
print(f"Registros con mejora en la predicción: {mejora} de {total}")
print(f"Promedio de mejora: {df_resultado['mejora_absoluta'].mean():.4f}")

# Mostrar el DataFrame resultante
print("Proceso completado. Resultados:")
display(df_resultado)



Evaluando el modelo Extra Trees:
Registros con mejora en la predicción: 258 de 600
Promedio de mejora: 1.0404
Proceso completado. Resultados:


,Alim_CuT,Alim_CuS,Alim CuI,Ag,Pb,Fe,Recuperacion_Turno,P80_Alim_Ro300,pH_Ro300,Tratamiento_Turno,Sol_Cit,Aire_Celdas,Nivel_Celdas,cumplimiento,encontro_contrafactual,recuperacion_predicha,prediccion_antes,prediccion_despues,mejora_absoluta
0,0.70,0.08,0.62,5.0,23.0,2.61,79.002079,209.0,10.400000,7827.943419,0.0500,109.206438,18.555931,0,no,NaN,80.108356,80.108356,0.000000
1,0.64,0.08,0.56,4.0,29.0,2.72,78.174603,228.0,11.200000,7863.186581,0.0800,109.041660,21.650116,0,no,NaN,75.572436,75.572436,0.000000
2,0.78,0.09,0.69,4.0,42.0,2.99,82.123510,198.0,11.343029,6781.529837,0.0800,95.063069,13.064303,0,si,80.275602,81.380634,80.275602,-1.105032
3,0.60,0.07,0.53,5.0,43.0,2.80,90.048983,183.0,11.500000,6011.236369,0.0600,117.918102,28.498127,1,no,NaN,81.641516,81.641516,0.000000
4,0.67,0.09,0.58,4.0,36.0,2.78,84.133993,218.0,11.200000,6420.382068,0.0700,118.092912,28.460662,0,no,NaN,81.785301,81.785301,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.91,0.07,0.84,6.0,120.0,3.67,87.775226,243.0,10.780000,9459.450000,0.0635,106.473936,9.634823,1,no,NaN,86.950392,86.950392,0.000000
596,0.94,0.09,0.85,5.0,71.0,3.60,90.174058,247.0,10.566667,7122.060000,0.0722,108.505663,15.414668,1,no,NaN,81.447754,81.447754,0.000000
597,0.72,0.06,0.66,6.0,65.0,2.76,86.434413,208.0,11.093333,8291.250000,0.0560,104.836123,14.108598,1,no,NaN,83.880040,83.880040,0.000000
598,0.80,0.06,0.74,5.0,228.0,3.19,89.483231,226.0,10.850000,7264.620000,0.0596,107.879839,13.586933,1,no,NaN,87.218480,87.218480,0.000000


In [69]:
df_resultado.to_csv('Resultado_test_3.csv', index=False)

In [ ]:
registro_original[Variables_predictoras].update(df_contrafactual.iloc[df_fila_menor_costo.index.values[0]])
display(f'el valor de recuperacion predicho con optimizacion es de {extra_trees_model.predict(registro_original[Variables_predictoras])}')
display(f'el valor de la recuperacion real es de {registro_original['Recuperacion_Turno'].values}')

## Simulación masiva (no hacer!!!)

In [ ]:
df_imputed['recuperacion_contrafactual']=0
df_imputed['ph_ro300_sugerido']=0
df_imputed['P80_Alim_Ro300_sugerido']=0
df_imputed['Tratamiento_turno_sugerido']=0
df_imputed['Alim_CuT_sugerido']=0

In [ ]:
df_imputed.info()

In [ ]:
# Selecciona una muestra específica donde 'cumplimiento' es 0 y elimina la columna 'cumplimiento'
# Itera sobre cada fila del DataFrame
for i in range(df_imputed.shape[0]):
    # Si la condición de cumplimiento es 0
    variables_drop=['Fecha', 'RECUPERACION_PONDERADA', 'Diferencia_Porcentual', 'Recuperacion_Calculada','recuperacion_contrafactual','ph_ro300_sugerido','P80_Alim_Ro300_sugerido','Tratamiento_turno_sugerido','Alim_CuT_sugerido','Recuperacion_rl','Recuperacion_rf']
    display(i)
    if model_rl.predict(df_imputed.iloc[i:i+1].drop(columns=variables_drop, axis=1)).item() < 86:
        # Selecciona la fila actual y elimina las columnas innecesarias
        registro = df_imputed.drop(columns=variables_drop, axis=1).iloc[i:i+1]
        #display(registro)
        # Genera contrafactuales para esta fila
        e = exp.generate_counterfactuals(
            registro,
            total_CFs=5,
            desired_class="opposite",
            permitted_range={
                'Alim CuT': [0.5, 0.9],
                'P80 Alim Ro300': [180, 210],
                'Tratamiento turno': [18000, 22000],
                'pH Ro300': [9, 11]
            },verbose=False

        )

        # Calcula los promedios de las columnas relevantes en el contrafactual
        c = e.cf_examples_list[0].final_cfs_df
        promedio_ph_ro300 = c['pH Ro300'].mean()
        promedio_Alim_CuT = c['Alim CuT'].mean()
        promedio_P80_Alim_Ro300 = c['P80 Alim Ro300'].mean()
        promedio_Tratamiento_turno = c['Tratamiento turno'].mean()

        # Actualiza 'registro' con los valores promedio del contrafactual
        registro_actualizado = registro.copy()
        registro_actualizado.loc[:, 'pH Ro300'] = promedio_ph_ro300
        registro_actualizado.loc[:, 'Alim CuT'] = promedio_Alim_CuT
        registro_actualizado.loc[:, 'P80 Alim Ro300'] = promedio_P80_Alim_Ro300
        registro_actualizado.loc[:, 'Tratamiento turno'] = promedio_Tratamiento_turno

        # Predice la recuperación con el modelo y asigna el resultado a la fila actual de 'recuperacion_contrafactual'
        df_imputed.loc[i, 'recuperacion_contrafactual'] = model_rl.predict(registro_actualizado).item()
        df_imputed.loc[i,'Alim_CuT_sugerido']=promedio_Alim_CuT
        df_imputed.loc[i,'ph_ro300_sugerido']=promedio_ph_ro300
        df_imputed.loc[i,'P80_Alim_Ro300_sugerido']=promedio_P80_Alim_Ro300
        df_imputed.loc[i,'Tratamiento_turno_sugerido']=promedio_Tratamiento_turno
    else:
        # Si 'cumplimiento' no es 0, asigna 'RECUPERACION_PONDERADA' a 'recuperacion_contrafactual'
        df_imputed.loc[i, 'recuperacion_contrafactual'] = model_rl.predict(registro).item()
        df_imputed.loc[i,'Alim_CuT_sugerido']=df_imputed.loc[i,'Alim CuT']
        df_imputed.loc[i,'ph_ro300_sugerido']=df_imputed.loc[i,'pH Ro300']
        df_imputed.loc[i,'P80_Alim_Ro300_sugerido']=df_imputed.loc[i,'P80 Alim Ro300']
        df_imputed.loc[i,'Tratamiento_turno_sugerido']=df_imputed.loc[i,'Tratamiento turno']


df_imputed.info()

In [ ]:
df_imputed.info()

In [ ]:
df_imputed.to_excel('Datos /df_imputed.xlsx', index=False)

In [ ]:
df_imputed['recuperacion_contrafactual'].head()

In [ ]:
sns.histplot(df_imputed['recuperacion_contrafactual'], kde=True)
plt.xlim(60,100)

In [ ]:
sns.histplot(df_imputed['RECUPERACION_PONDERADA'].head(171), kde=True)
plt.xlim(60,100)